In [2]:
!pip install scikit-surprise




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357255 sha256=0ba6cc0a149b0268b7a6482aeebedef23a06fc583831286416f2de39e7088cd3
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [3]:
from surprise import SVD, Dataset, accuracy, SVDpp, NMF
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV

In [4]:
data= Dataset.load_builtin('ml-100k')


Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [5]:
pram_svd={'n_epochs':[5,10,15],
          'lr_all':[0.002,0.005,0.01],
          'reg_all':[0.02,0.05,0.1]}
pram_svdpp={'n_epochs':[5,10,15],
            'lr_all':[0.002,0.005,0.01],
            'reg_all':[0.02,0.05,0.1]}
pram_nmf={'n_epochs':[5,10,15],
          'n_factors':[10,20,50],
          'reg_pu':[0.002,0.005,0.01],
          'reg_qi':[0.02,0.05,0.1]}

In [6]:
gir_svd= GridSearchCV(SVD, pram_svd, measures=[ 'rmse'], cv=3)
gir_svd.fit(data)
best_s_parms= gir_svd.best_params['rmse']
best_s_score= gir_svd.best_score['rmse']
print(best_s_parms)
print(best_s_score)

{'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.05}
0.9316294783232797


In [7]:
gir_svdpp= GridSearchCV(SVDpp, pram_svdpp, measures=['rmse'], cv=3)
gir_svdpp.fit(data)
best_spp_parms= gir_svdpp.best_params['rmse']
best_spp_score= gir_svdpp.best_score['rmse']
print(best_spp_score)
print(best_spp_score)

0.9263313393731037
0.9263313393731037


In [8]:
gir_nmf= GridSearchCV(NMF, pram_nmf, measures=['rmse'], cv=3)
gir_nmf.fit(data)
best_nmf_parms= gir_nmf.best_params['rmse']
best_nmf_score= gir_nmf.best_score['rmse']
print(best_nmf_parms)
print(best_nmf_score)

{'n_epochs': 15, 'n_factors': 10, 'reg_pu': 0.01, 'reg_qi': 0.1}
1.1040553806735802


In [9]:
if best_s_score < best_spp_score and best_s_score < best_nmf_score:
    best_algorytm= SVD(**best_s_parms)
elif best_spp_score < best_s_score and best_spp_score < best_nmf_score:
    best_algorytm= SVDpp(**best_spp_parms)
elif best_nmf_score < best_s_score and best_nmf_score <best_spp_score:
    best_algorytm= NMF(**best_nmf)
else:
    print("Brak odpowiedniego algorytmu")

In [12]:
trainset, testset= train_test_split(data, test_size=0.25)
best_algorytm.fit(trainset)
predictions= best_algorytm.test(testset)
finnly_algorytm=accuracy.rmse(predictions)
print({finnly_algorytm})

RMSE: 0.9236
{0.9235955731094014}
